<a href="https://colab.research.google.com/github/vigneshwaran-dev/CV-research-timeline/blob/main/AlexNet/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, Lambda
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.nn import local_response_normalization
from tensorflow.keras.optimizers import SGD

Defining the Model as per the Original Paper

In [3]:
model = Sequential();

# 1st Convolutional Layer
model.add(Conv2D(input_shape=(224, 224, 3), filters=96, kernel_size=(11, 11), strides=(4, 4), padding='same'))
model.add(Lambda(local_response_normalization)) 
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same'))

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(5, 5), padding='same'))
model.add(Lambda(local_response_normalization))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same'))

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3, 3), padding='same'))
model.add(Activation('relu'))

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3, 3), padding='same'))
model.add(Activation('relu'))

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3, 3), padding='same'))
model.add(Activation('relu'))

# 1st Dense Layer
model.add(Flatten())
model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# 2nd Dense Layer
model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(1000))
model.add(Activation('softmax'))

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 56, 56, 96)        34944     
_________________________________________________________________
lambda (Lambda)              (None, 56, 56, 96)        0         
_________________________________________________________________
activation (Activation)      (None, 56, 56, 96)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 28, 28, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 256)       614656    
_________________________________________________________________
lambda_1 (Lambda)            (None, 28, 28, 256)       0         
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 256)       0

In [5]:
model.compile(loss=categorical_crossentropy,
                 optimizer=SGD(learning_rate=0.01),
                 metrics=['accuracy'])

Considering the data to be present in TRAIN_DATA_LOCATION and VALIDATION_DATA_LOCATION directories and running them through data generators to perform live data augumentation during the training process

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = 'TRAIN_DATA_LOCATION'
valid_dir = 'VALIDATION_DATA_LOCATION'

BATCH_SIZE = 32

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=10,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.1,
                                   zoom_range=0.1)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(224, 224),
                                                    color_mode='rgb',
                                                    batch_size=BATCH_SIZE,
                                                    seed=1,
                                                    shuffle=True,
                                                    class_mode='categorical')

valid_datagen = ImageDataGenerator(rescale=1.0/255.0)

valid_generator = valid_datagen.flow_from_directory(valid_dir,
                                                    target_size=(224, 224),
                                                    color_mode='rgb',
                                                    batch_size=BATCH_SIZE,
                                                    seed=7,
                                                    shuffle=True,
                                                    class_mode='categorical')

train_num = train_generator.samples

Training the Model

In [ ]:
import datetime

log_dir = 'logs/fit/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
callback_list = [tensorboard_callback]

model.fit(train_generator,
          epochs=1,
          steps_per_epoch=train_num // BATCH_SIZE,
          validation_data=valid_generator,
          validation_steps=valid_num // BATCH_SIZE,
          callbacks=callback_list,
          verbose=1)

model.save('alexnet.h5')

Visualizing the performance using Tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/fit

Prediction

In [ ]:
x_valid, label_batch  = next(iter(valid_generator))
prediction_values = model.predict_classes(x_valid)
print(prediction_values)